<a href="https://colab.research.google.com/github/SnrPep/Mos/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install pymorphy3
import nltk
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import re
from math import log
import numpy as np

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
stop_words = set(stopwords.words('russian'))

morph = MorphAnalyzer()

texts = [
    "Мама мыла раму, а потом пошла гулять в парк.",
    "Папа чинил машину, а мама готовила ужин.",
    "Дети играли в парке, а родители отдыхали."
]

def preprocess_text(text):
    # Токенизация
    words = re.findall(r'\b\w+\b', text.lower())

    # Лемматизация
    lemmas = [morph.parse(word)[0].normal_form for word in words if word not in stop_words and word.isalpha()]

    return lemmas

processed_texts = [preprocess_text(text) for text in texts]
print(processed_texts)

# Словарь уникальных слов)
vocabulary = set()
for text in processed_texts:
    vocabulary.update(text)
vocabulary = sorted(vocabulary)
vocabulary_list = list(vocabulary)

[['мама', 'мыло', 'рама', 'пойти', 'гулять', 'парк'], ['папа', 'чинить', 'машина', 'мама', 'готовить', 'ужин'], ['ребёнок', 'играть', 'парка', 'родитель', 'отдыхать']]


In [30]:
# Bag of Words
def manual_bow(texts, vocabulary):
    bow_matrix = []
    for text in texts:
        word_counts = {word: 0 for word in vocabulary}
        for word in text:
            if word in word_counts:
                word_counts[word] += 1
        bow_matrix.append([word_counts[word] for word in vocabulary])
    return np.array(bow_matrix)

bow_matrix = manual_bow(processed_texts, vocabulary_list)
print("Матрица Bag of Words:")
print(bow_matrix)

Матрица Bag of Words:
[[0 1 0 1 0 1 0 0 1 0 1 1 0 0 0 0]
 [1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1]
 [0 0 1 0 0 0 1 0 0 1 0 0 1 1 0 0]]


In [31]:
# TF-IDF
def manual_tfidf(bow_matrix):
    # TF
    tf = bow_matrix / bow_matrix.sum(axis=1, keepdims=True)

    # IDF
    idf = np.array([log(len(bow_matrix) / (1 + np.count_nonzero(bow_matrix[:, i]))) for i in range(bow_matrix.shape[1])])

    # TF-IDF
    tfidf = tf * idf
    return tfidf

tfidf_matrix = manual_tfidf(bow_matrix)
print("Матрица TF-IDF:")
print(tfidf_matrix)


Матрица TF-IDF:
[[0.         0.06757752 0.         0.         0.         0.06757752
  0.         0.         0.06757752 0.         0.06757752 0.06757752
  0.         0.         0.         0.        ]
 [0.06757752 0.         0.         0.         0.06757752 0.
  0.         0.06757752 0.         0.         0.         0.
  0.         0.         0.06757752 0.06757752]
 [0.         0.         0.08109302 0.         0.         0.
  0.08109302 0.         0.         0.08109302 0.         0.
  0.08109302 0.08109302 0.         0.        ]]
